In [1]:
import pandas as pd
from glob import glob
#import winsound
#import sklearn.model_selection as model_selection
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
#from microtc.utils import save_model

fnames = glob("datasets//*_train.json")
#fnames = glob("datasets/meoffendes2021_mx_Es_train.json")
fnames.sort()

P = []
for fn in fnames:

    ds_n = fn.replace("datasets\\",'').replace('_train.json','')
    print('__', ds_n, '___')

    train_df = pd.read_json(fn, lines=True)
    test_df = pd.read_json(fn.replace("_train", "_test"), lines=True)

    # recortados para pruebas
    #train_df = train_df.loc[0:399, ['text', 'klass']]
    #test_df = test_df.loc[0:79, ['text', 'klass']]
    
    if False:  # True para BiFrases
        """
        TODO: en cada append, el dataset se reconstruye completo. Encontrar otro método más rápido.
        """
        _ = train_df.iloc[0]
        bi_train_df = pd.DataFrame([_], columns = ['text', 'klass'])
        for index, row in train_df.iloc[1:].iterrows():
            try:
                bi_train_df =  bi_train_df.append(row)
                if _['klass'] == row['klass']:
                    bi_frase = {'text': _['text'] + ' ' + row['text'], 'klass':row['klass']}
                    bi_train_df = bi_train_df.append(bi_frase, ignore_index=True)
                    _ =  {'text': '_V_A_C_I_A_', 'klass': None}
                else:   
                    _ = row
            except Exception as exc:
                print(exc)
                _ = row
                continue
        train_df = bi_train_df
        #winsound.Beep(3000, 900)
    
    """
    TODO: No estoy usando el dataset test; en su lugar, hago el split del train. No he visto como
    convertir el df en la matrix que pide el MNB
    """
    
    #X_train = train_df[['text']]
    #y_train = train_df[['klass']]
    
    #X_test = test_df[['text']]
    #y_test = test_df[['klass']]
    
    
    token = RegexpTokenizer(r'[a-zA-Z0-9]+')
    cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
    text_counts = cv.fit_transform(train_df['text']) 

    X_train, X_test, y_train, y_test = train_test_split(text_counts, train_df['klass'], test_size=0.25, random_state=5)
    
    MNB = MultinomialNB()
    MNB.fit(X_train, y_train)
    
    predicted = MNB.predict(X_test)
    
    f1_score = metrics.f1_score(predicted, y_test, average="macro")
    precision_score = metrics.precision_score(predicted, y_test, average="macro")
    recall_score = metrics.recall_score(predicted, y_test, average="macro")
    f1_score_micro = metrics.recall_score(predicted, y_test, average="micro")
    
    p = [ds_n, f1_score, precision_score, recall_score, f1_score_micro] 
    P.append(p)
    
    #winsound.Beep(3000, 900)
    
    #break

#save_model(P, 'MNB_BiFrases_True.scores')    
    
#winsound.Beep(2500, 1600) 

print('dataset,', 'f1_macro,', 'precision_macro,', 'recall_macro,', 'f1_micro')
for p in P:
    print("{0}, {1:1.5f}, {2:1.5f}, {3:1.5f}, {4:1.5f}".format(p[0], p[1], p[2], p[3], p[4]))
    

__ SCv1_En ___
__ SS-Twitter_En ___
__ SS-Youtube_En ___
__ business_En ___
__ haha2018_Es ___
__ meoffendes2021_mx_Es ___
__ mexa3t2018_aggress_Es ___
__ misoginia_Es ___
__ semeval2018_anger_Es ___
dataset, f1_macro, precision_macro, recall_macro, f1_micro
SCv1_En, 0.59556, 0.59698, 0.59698, 0.59556
SS-Twitter_En, 0.72454, 0.72011, 0.73552, 0.74222
SS-Youtube_En, 0.74462, 0.73067, 0.77567, 0.79111
business_En, 0.71601, 0.73056, 0.73889, 0.77273
haha2018_Es, 0.77383, 0.77641, 0.77169, 0.78900
meoffendes2021_mx_Es, 0.68582, 0.67092, 0.72917, 0.76877
mexa3t2018_aggress_Es, 0.75782, 0.75420, 0.76264, 0.77893
misoginia_Es, 0.73260, 0.73268, 0.73260, 0.73263
semeval2018_anger_Es, 0.31228, 0.31146, 0.38754, 0.44706


In [14]:
for p in P:
    print("{0}, {1:1.5f}, {2:1.5f}, {3:1.5f}, {4:1.5f}".format(p[0], p[1], p[2], p[3], p[4]))

meoffendes2021_mx_Es, 0.68582, 0.67092, 0.72917, 0.76877
